In [16]:
# index_event_backtest.py
import pandas as pd
import numpy as np
import datetime as dt
from dataclasses import dataclass
import yfinance as yf
from typing import Dict, List, Tuple, Optional

In [18]:
# ---------- Config ----------
GROSS_CAP = 5_000_000     # total gross exposure cap
TXN_COST_PER_SHARE = 0.01 # $/share each side
OPEN = "Open"
CLOSE = "Close"

@dataclass
class Rates:
    # daily interest rates (simple, not compounding intra-day)
    # pass as decimals per day (e.g., annual_rate/252)
    long_daily: float   # Fed Funds + 1.5% annualized / 252
    short_daily: float  # 1.0% annualized / 252

@dataclass
class Params:
    adv_cap: float = 0.01        # ≤ 1% of 20-day ADV
    use_spy_hedge: bool = True   # hedge choice
    hold_max_days: int = 22      # up to 1 month (≈ 22 trading days)
    exit_on_trade_date: bool = True
    size_method: str = "equal_cash"  # or "by_adv"
    outperf_window: int = 1      # intraday rule check window for Strategy 2

In [19]:
# Normalizing Column names and adding "is_net_buy" flag
def read_events(path: str, sheet: Optional[str] = None) -> pd.DataFrame:
    """Load your events table. Excel or CSV ok. Expect columns at least:
       ticker, announce_dt, trade_dt, shares_to_trade, index_name, event_type
       Optional: is_net_buy (or compute from shares_to_trade > 0)
    """
    if path.lower().endswith((".xlsx", ".xls")):
        df = pd.read_excel(path, sheet_name=sheet)
    else:
        df = pd.read_csv(path)
    # normalize columns
    ren = {
        "Ticker":"ticker",
        "Announced":"announce_dt",
        "TradeDate":"trade_dt",
        "Index Change":"index_name",
        "Action":"event_type",
        "Shs to Trade":"shares_to_trade",
    }
    df = df.rename(columns={k:v for k,v in ren.items() if k in df.columns})
    # parse dates
    for c in ["announce_dt","trade_dt"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c]).dt.date
    if "is_net_buy" not in df.columns and "shares_to_trade" in df.columns:
        df["is_net_buy"] = (df["shares_to_trade"].fillna(0) > 0).astype(int)
    # canonical subset
    keep = ["ticker","announce_dt","trade_dt","index_name","event_type",
            "shares_to_trade","is_net_buy"]
    return df[keep].dropna(subset=["ticker","announce_dt","trade_dt"]).copy()


In [23]:
def get_price_history(tickers: List[str], start: str, end: str) -> Dict[str, pd.DataFrame]:
    """Download OHLCV from yfinance. Returns dict[ticker]->DataFrame(date-indexed)."""
    out = {}
    for t in sorted(set(tickers)):
        data = yf.download(t, start=start, end=end, auto_adjust=False, progress=False)
        if data.empty: 
            continue
        data.index = data.index.tz_localize(None)
        out[t] = data[["Open","Close","Adj Close","Volume"]].copy()
    return out

In [24]:
def get_index_history(index_map: Dict[str,str], start: str, end: str) -> Dict[str, pd.DataFrame]:
    """index_map example: {'S&P 500':'^GSPC','Russell 2000':'^RUT'}"""
    out = {}
    for name, ticker in index_map.items():
        data = yf.download(ticker, start=start, end=end, auto_adjust=False, progress=False)
        if data.empty:
            continue
        data.index = data.index.tz_localize(None)
        out[name] = data[["Open","Close","Adj Close"]].copy()
    return out

In [25]:
def trading_days_union(price_dict: Dict[str, pd.DataFrame]) -> List[pd.Timestamp]:
    ds = None
    for df in price_dict.values():
        ds = df.index if ds is None else ds.union(df.index)
    return list(pd.DatetimeIndex(ds).sort_values())

def compute_adv20(prices: Dict[str,pd.DataFrame]) -> Dict[str, pd.Series]:
    """Compute 20-day ADV using Close*Volume rolling mean."""
    adv = {}
    for t, df in prices.items():
        dv = (df["Close"] * df["Volume"]).rolling(20, min_periods=5).mean()
        adv[t] = dv
    return adv

def open_after(date_: dt.date, cal: List[pd.Timestamp]) -> Optional[pd.Timestamp]:
    """Next trading day open timestamp."""
    for d in cal:
        if d.date() > date_:
            return d
    return None

def clip_by_adv(shares_target: float, adv_usd: float, price: float, cap: float) -> int:
    """Cap shares so that notional ≤ cap * ADV (USD)."""
    if np.isnan(adv_usd) or adv_usd <= 0 or np.isnan(price) or price <= 0:
        return 0
    max_notional = cap * adv_usd
    max_shares = int(max_notional // price)
    return int(np.sign(shares_target) * min(abs(int(shares_target)), max_shares))

def size_positions(candidates: List[Tuple[str,int,float]], gross_cap: float, method: str="equal_cash"):
    """
    candidates: list of (ticker, side, price) where side=+1 long, -1 short
    Returns dict[ticker]->target_shares ignoring ADV cap (that comes later).
    """
    if not candidates:
        return {}
    n = len(candidates)
    if method == "equal_cash":
        cash_per_leg = gross_cap / n
        return {t: int(cash_per_leg // p) * s for (t,s,p) in candidates if p > 0}
    else:
        # extend with ADV-based weights if you pass adv info here
        cash_per_leg = gross_cap / n
        return {t: int(cash_per_leg // p) * s for (t,s,p) in candidates if p > 0}

def daily_financing(notional_long: float, notional_short: float, rates: Rates) -> float:
    """Apply daily financing cost (negative PnL)."""
    return notional_long * rates.long_daily + abs(notional_short) * rates.short_daily

def txn_cost(shares: int) -> float:
    return abs(shares) * TXN_COST_PER_SHARE

In [26]:
# ---------- Strategy Logic ----------
def build_trades_strategy1(events: pd.DataFrame,
                           prices: Dict[str,pd.DataFrame],
                           adv: Dict[str,pd.Series],
                           params: Params) -> List[dict]:
    """
    Strategy 1:
    - After announcement, enter at next open.
    - Direction: sign(shares_to_trade): net buy => long, net sell => short.
    - Exit any day before or on trade date (we’ll default to trade date close).
    - Enforce ≤1% ADV (USD) and gross cap.
    """
    cal = trading_days_union(prices)
    trades = []
    # batch sizing per day
    grouped = events.groupby("announce_dt")
    for adate, grp in grouped:
        # identify entries (next open)
        entries = []
        for _, row in grp.iterrows():
            t = row["ticker"]
            if t not in prices: 
                continue
            ent_day = open_after(row["announce_dt"], cal)
            if ent_day is None: 
                continue
            side = +1 if row["shares_to_trade"] > 0 else -1
            ent_px = prices[t].loc[ent_day, OPEN] if ent_day in prices[t].index else np.nan
            if np.isnan(ent_px): 
                continue
            entries.append((t, side, float(ent_px), row["trade_dt"]))
        # size equally
        sizes = size_positions(entries, GROSS_CAP, params.size_method)
        # create trades with ADV cap and exit on trade date close (default)
        for (t, side, ent_px, trade_dt) in entries:
            dfp = prices[t]
            # pick exit day: min(trade_dt, ent_day + hold_max_days)
            ent_idx = dfp.index.get_indexer([ent_day], method="nearest")[0]
            max_exit_idx = min(ent_idx + params.hold_max_days, len(dfp)-1)
            # default exit date = trade date (close), but must be on calendar
            exit_day = None
            # nearest on/just before trade_dt
            idx_on = dfp.index.get_indexer([pd.Timestamp(trade_dt)], method="pad")[0] if len(dfp.index[dfp.index.date<=trade_dt]) else None
            if idx_on is not None:
                exit_day = dfp.index[min(idx_on, max_exit_idx)]
            else:
                exit_day = dfp.index[max_exit_idx]
            exit_px = dfp.loc[exit_day, CLOSE]
            # ADV cap at entry
            adv_usd = adv[t].loc[ent_day] if t in adv and ent_day in adv[t].index else np.nan
            tgt_shares = sizes.get(t, 0) * side
            capped_shares = clip_by_adv(tgt_shares, adv_usd, ent_px, params.adv_cap)
            if capped_shares == 0: 
                continue
            trades.append({
                "strategy":"S1",
                "ticker":t,
                "entry_dt": ent_day.date(),
                "entry_px": float(ent_px),
                "exit_dt": exit_day.date(),
                "exit_px": float(exit_px),
                "shares": int(capped_shares),
            })
    return trades

In [27]:
def outperforming(stock_df: pd.DataFrame, index_df: pd.DataFrame, day: pd.Timestamp) -> bool:
    """Stock outperforming index intraday (Open->Close) on 'day'."""
    if day not in stock_df.index or day not in index_df.index:
        return False
    s_ret = stock_df.loc[day, CLOSE] / stock_df.loc[day, OPEN] - 1.0
    i_ret = index_df.loc[day, CLOSE] / index_df.loc[day, OPEN] - 1.0
    return s_ret > i_ret

def build_trades_strategy2(events: pd.DataFrame,
                           prices: Dict[str,pd.DataFrame],
                           index_prices: Dict[str,pd.DataFrame],
                           params: Params) -> List[dict]:
    """
    Strategy 2 (intraday on trade date):
    - Short on the “trade date” when:
        a) event is net buy (shares_to_trade > 0)
        b) stock is outperforming its index (Open->Close) on trade date
    - Entry: trade date open
    - Exit: next day close by default (tune holding horizon).
    """
    cal = trading_days_union(prices)
    trades = []
    for _, row in events.iterrows():
        if row["shares_to_trade"] <= 0: 
            continue  # only net buys
        t = row["ticker"]; idx_name = row.get("index_name", None)
        if t not in prices or idx_name not in index_prices: 
            continue
        trade_dt = row["trade_dt"]
        # ensure trade date is a trading day for the stock
        s_df = prices[t]
        i_df = index_prices[idx_name]
        # pick nearest trading day ON/PAD to trade_dt
        idx = s_df.index.get_indexer([pd.Timestamp(trade_dt)], method="pad")
        if len(idx)==0 or idx[0] < 0: 
            continue
        day = s_df.index[idx[0]]
        # condition: outperforming on the day
        if not outperforming(s_df, i_df, day):
            continue
        # build short
        ent_px = s_df.loc[day, OPEN]
        # exit day: next day close (or cap by hold_max_days)
        ent_idx = s_df.index.get_loc(day)
        exit_idx = min(ent_idx + 1, min(ent_idx + params.hold_max_days, len(s_df)-1))
        exit_day = s_df.index[exit_idx]
        exit_px = s_df.loc[exit_day, CLOSE]
        # simple equal sizing; ADV cap at entry
        sizes = size_positions([(t,-1,ent_px)], GROSS_CAP, params.size_method)
        tgt_shares = sizes.get(t, 0) * -1
        adv_usd = (s_df["Close"]*s_df["Volume"]).rolling(20, min_periods=5).mean().get(day, np.nan)
        capped_shares = clip_by_adv(tgt_shares, adv_usd, ent_px, params.adv_cap)
        if capped_shares == 0: 
            continue
        trades.append({
            "strategy":"S2",
            "ticker":t,
            "entry_dt": day.date(),
            "entry_px": float(ent_px),
            "exit_dt": exit_day.date(),
            "exit_px": float(exit_px),
            "shares": int(capped_shares),
        })
    return trades

In [28]:
# ---------- PnL Engine ----------
def apply_txn_costs(trades: List[dict]) -> None:
    for tr in trades:
        tr["cost_entry"] = txn_cost(tr["shares"])
        tr["cost_exit"]  = txn_cost(tr["shares"])

def attach_daily_pnl(trades: List[dict],
                     prices: Dict[str,pd.DataFrame],
                     rates: Rates,
                     use_spy_hedge: bool = True) -> pd.DataFrame:
    """
    Expands trades to daily PnL with financing + optional SPY hedge.
    """
    # Build daily rows
    rows = []
    for tr in trades:
        t = tr["ticker"]
        if t not in prices: 
            continue
        dfp = prices[t]
        d0 = pd.Timestamp(tr["entry_dt"])
        d1 = pd.Timestamp(tr["exit_dt"])
        held = dfp.loc[(dfp.index.date>=d0.date()) & (dfp.index.date<=d1.date())].copy()
        if held.empty: 
            continue
        sh = tr["shares"]
        # entry fill at open on d0; exit at close on d1
        entry_open = dfp.loc[dfp.index.date==d0.date(), OPEN].iloc[0]
        exit_close = dfp.loc[dfp.index.date==d1.date(), CLOSE].iloc[0]
        # accumulate daily MTM
        prev_close = entry_open
        for day, row in held.iterrows():
            close_px = row["Close"]
            day_ret = (close_px - prev_close) * sh        # $ MTM vs prior close
            notional_long = max(sh*prev_close, 0)
            notional_short = min(sh*prev_close, 0)
            fin_cost = -daily_financing(notional_long, notional_short, rates)
            rows.append({
                "date": day.date(),
                "ticker": t,
                "daily_mtm": float(day_ret),
                "financing": float(fin_cost),
                "shares": int(sh),
                "close_px": float(close_px),
            })
            prev_close = close_px
        # overwrite final day realized to match exit at close; entry/exit costs
        rows[-1]["daily_mtm"] += -txn_cost(sh)  # charge exit on last day
        # charge entry on first day (find first index for this trade)
        for i in range(len(rows)):
            if rows[i]["ticker"]==t and rows[i]["date"]==d0.date():
                rows[i]["daily_mtm"] += -txn_cost(sh)
                break
    pnl = pd.DataFrame(rows)
    if pnl.empty:
        return pnl
    pnl["daily_pnl"] = pnl["daily_mtm"] + pnl["financing"]
    return pnl

In [29]:
# ---------- Driver ----------
def run_backtest(events_path: str,
                 index_map: Dict[str,str],
                 start: str,
                 end: str,
                 fed_funds_annual: float = 0.055,   # e.g., 5.5% (placeholder)
                 add_long_spread: float = 0.015,    # +1.5%
                 short_borrow_annual: float = 0.01, # 1%
                 sheet: Optional[str] = None,
                 params: Params = Params()) -> Dict[str, pd.DataFrame]:

    events = read_events(events_path, sheet)
    tickers = events["ticker"].dropna().unique().tolist()

    prices = get_price_history(tickers, start, end)
    index_prices = get_index_history(index_map, start, end)
    adv20 = compute_adv20(prices)

    # convert annualized to daily (252 trading days)
    rates = Rates(long_daily=(fed_funds_annual + add_long_spread)/252.0,
                  short_daily=(short_borrow_annual)/252.0)

    s1_trades = build_trades_strategy1(events, prices, adv20, params)
    s2_trades = build_trades_strategy2(events, prices, index_prices, params)

    apply_txn_costs(s1_trades); apply_txn_costs(s2_trades)

    s1_pnl = attach_daily_pnl(s1_trades, prices, rates, params.use_spy_hedge)
    s2_pnl = attach_daily_pnl(s2_trades, prices, rates, params.use_spy_hedge)

    # summaries
    def summarize(df):
        if df.empty: return pd.DataFrame()
        out = df.groupby("date")["daily_pnl"].sum().to_frame("pnl")
        out["cum_pnl"] = out["pnl"].cumsum()
        out["return_bp_on_5mm"] = out["pnl"]/GROSS_CAP*10_000
        return out

    return {
        "events": events,
        "s1_trades": pd.DataFrame(s1_trades),
        "s2_trades": pd.DataFrame(s2_trades),
        "s1_pnl_daily": summarize(s1_pnl),
        "s2_pnl_daily": summarize(s2_pnl),
    }

if __name__ == "__main__":
    # Example usage:
    # 1) Put your events at ./index_events_cleaned.csv (or your Excel + sheet name)
    # 2) Choose start/end buffers around your event dates
    idx_map = {
        "S&P 500": "^GSPC",
        "Russell 2000": "^RUT",
        "Russell 1000": "^RUI",
        "NASDAQ 100": "^NDX",
    }
    results = run_backtest(
        events_path="Index Event Data.xlsx",  # or "Index Event Data.xlsx"
        index_map=idx_map,
        start="2022-04-01",
        end="2025-08-28",
        fed_funds_annual=0.055,  # replace with actual FRED series
        sheet=None,               # set sheet name if using Excel
        params=Params(
            adv_cap=0.01, use_spy_hedge=True, hold_max_days=22,
            exit_on_trade_date=True, size_method="equal_cash"
        )
    )
    # Save outputs
    for k, v in results.items():
        if isinstance(v, pd.DataFrame) and not v.empty:
            v.to_csv(f"{k}.csv", index=True)
            print(f"Wrote {k}.csv")

AttributeError: 'dict' object has no attribute 'rename'